In [4]:

import datetime
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse_fraction, contri, get_tensor, create_region_df_dfc_static
from create_matrix import *
from tensor_custom_core_all import *
import multiprocessing as mp

global source, target
global case
global source_df, source_dfc, source_tensor, source_static
global target_df, target_dfc, target_tensor, target_static
global source_L, target_L
global T_constant

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}
APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
year = 2014
case = 2
source = 'Austin'
constant_use = True
target = 'SanDiego'
A_store = pickle.load(open(os.path.expanduser('~/git/scalable-nilm/aaai18/predictions/case-{}-graph_{}_{}_all_As.pkl'.format(case, source, constant_use)), 'r'))
source_df, source_dfc, source_tensor, source_static = create_region_df_dfc_static(source, year)
target_df, target_dfc, target_tensor, target_static = create_region_df_dfc_static(target, year)

# # using cosine similarity to compute L
source_L = get_L(source_static)
target_L = get_L(target_static)


T_constant = np.ones(12).reshape(-1 , 1)


pred = {}
n_splits = 10

for appliance in APPLIANCES_ORDER:
    pred[appliance] = []



In [5]:

kf = KFold(n_splits=10)


count = 0
error = []
params = {}
H_factors = {}

In [6]:
learning_rate_cv = 0.1
num_season_factors_cv = 2
num_home_factors_cv = 3
lam_cv = 0.001
num_iterations_cv = 1300

In [10]:
A_source = A_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv][lam_cv][num_iterations_cv]

random_seed = 0
train_percentage = 10

for outer_loop_iteration, (train_max, test) in enumerate(kf.split(target_df)):
    # Just a random thing
    print num_iterations_cv, num_season_factors_cv, num_home_factors_cv, lam_cv
    np.random.seed(10 * random_seed + 7 * outer_loop_iteration)
    np.random.shuffle(train_max)
    print("-" * 80)
    print("Progress: {}".format(100.0 * outer_loop_iteration / n_splits))
    print(datetime.datetime.now())
    sys.stdout.flush()
    num_train = int((train_percentage * len(train_max) / 100) + 0.5)


    if train_percentage == 100:
        train = train_max
        train_ix = target_df.index[train]
        # print("Train set {}".format(train_ix.values))
        test_ix = target_df.index[test]
    else:
        # Sample `train_percentage` homes
        # An important condition here is that all homes should have energy data
        # for all appliances for atleast one month.
        train, _ = train_test_split(train_max, train_size=train_percentage / 100.0)
        train_ix = target_df.index[train]
        test_ix = target_df.index[test]
    print train_ix

    print("-" * 80)
    print("Test set {}".format(test_ix.values))
    print("-" * 80)
    print("Current Error, Least Error, #Iterations")

    num_test = len(test_ix)
    train_test_ix = np.concatenate([test_ix, train_ix])
    df_t, dfc_t = target_df.loc[train_test_ix], target_dfc.loc[train_test_ix]
    tensor = get_tensor(df_t)
    tensor_copy = tensor.copy()
    # First n

    L = target_L[np.ix_(np.concatenate([test, train]), np.concatenate([test, train]))]

    H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy, L,
                                                              num_home_factors_cv,
                                                              num_season_factors_cv,
                                                              num_iter=num_iterations_cv, lr=learning_rate_cv, dis=False,
                                                              lam=lam_cv, A_known=A_source, T_known=T_constant)

    HAT = multiply_case(H, A, T, case)
    for appliance in APPLIANCES_ORDER:
        pred[appliance].append(pd.DataFrame(HAT[:num_test, appliance_index[appliance], :], index=test_ix))



    

1300 2 3 0.001
--------------------------------------------------------------------------------
Progress: 0.0
2017-09-06 22:17:32.538273
Int64Index([9775, 9370, 5909], dtype='int64')
--------------------------------------------------------------------------------
Test set [  54  203  527 1450]
--------------------------------------------------------------------------------
Current Error, Least Error, #Iterations
1300 2 3 0.001
--------------------------------------------------------------------------------
Progress: 10.0
2017-09-06 22:17:34.062679
Int64Index([54, 9612, 2606], dtype='int64')
--------------------------------------------------------------------------------
Test set [1524 1731 2031 2354]
--------------------------------------------------------------------------------
Current Error, Least Error, #Iterations
1300 2 3 0.001
--------------------------------------------------------------------------------
Progress: 20.0
2017-09-06 22:17:35.561630
Int64Index([7062, 2354, 4095], 

In [75]:
s = pd.concat(pred[appliance]).ix[target_df.index]
err = {}
for appliance in APPLIANCES_ORDER[1:]:
    if appliance=="hvac":
        err[appliance] = compute_rmse_fraction(appliance,s[range(4, 10)], target)[3]
    else:   
        err[appliance] = compute_rmse_fraction(appliance, s,target)[3]

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [76]:
err

{'dw': dw_1   1450    0.570397
        1731    1.887388
        2606    2.554559
        3687    2.435172
        3938    2.955308
        4329    0.744343
        4495    1.555951
        5938    1.783778
        6268    0.035106
        6377    5.367970
        7062    0.105135
        8342    2.680484
        8574    0.937927
        9213    0.209764
        9775    2.275840
 dw_10  203     3.128731
        1450    1.112111
        1524    0.698513
        1731    1.659378
        2354    1.348476
        2606    1.525120
        3687    1.935588
        3864    1.357384
        3938    1.310617
        4495    1.297457
        4934    1.404787
        5938    4.050695
        6377    1.868008
        6429    4.272922
        6547    3.098096
                  ...   
 dw_9   203     1.567447
        527     0.214655
        1450    0.487781
        1524    0.328798
        1731    0.801144
        2354    0.471398
        2606    0.836123
        3687    1.407653
        3864    0.7

In [77]:
error = pd.concat([err['hvac']['hvac_5'], err['hvac']['hvac_6']], axis=1)

In [78]:
k = {}
for appliance in APPLIANCES_ORDER[1:]:
    
    if appliance == 'hvac':
        start, end = 5, 11
    else:
        start, end = 1, 13
    error = pd.concat([err[appliance][appliance + "_{}".format(start)], 
                       err[appliance][appliance + "_{}".format(start+1)]],axis=1)
    
    for i in range(start+2, end):
        error = pd.concat([error, err[appliance][appliance + "_{}".format(i)]], axis = 1)
    app = np.sqrt((error**2).mean(axis=1))
    k[appliance] = app

In [79]:
pd.concat(k, axis=1)

,dw,fridge,hvac,mw,oven,wm
54,2.713467,9.572421,25.818999,0.626947,NaN,0.777124
203,2.255839,8.811615,31.527562,0.360641,1.461839,NaN
527,0.155368,1.980107,48.596957,0.734986,0.544021,NaN
1450,1.051524,7.078746,29.018152,1.004387,1.864285,NaN
1524,1.254952,9.577246,41.015785,1.845209,0.806821,NaN
1731,1.854061,5.405594,26.451095,1.708080,1.118277,NaN
2031,3.592711,12.847996,19.691791,2.166383,NaN,2.275351
2354,1.276594,NaN,28.270201,1.535626,NaN,0.489230
2606,2.168367,NaN,39.442029,1.260919,1.587481,NaN
3687,2.247050,12.231422,26.484160,1.169503,1.030089,NaN


In [105]:
result = (pd.DataFrame(k).fillna(0)*pd.Series(contri['SanDiego'])).sum(axis=1)

In [107]:
result

54      17.454664
203     20.397646
527     27.642747
1450    18.467971
1524    25.910668
1731    16.543539
2031    15.206043
2354    15.799331
2606    22.063175
3687    18.690229
3864    24.948328
3938    12.250179
4083    16.543785
4095    20.022487
4329    18.074604
4495     8.084136
4761    11.572906
4934    17.924379
5909     4.660579
5938    10.855711
6268     2.407033
6377    12.462037
6429    22.354747
6497    17.302611
6547    15.069861
7062     4.203243
7114    22.335949
7409    23.770803
7639    15.416784
8061    23.801426
8342    14.239676
8574    11.296507
8733    16.608344
9213    16.916956
9370    26.981516
9585    22.398928
9612    27.973157
9775    10.637744
9836    13.627245
dtype: float64

In [106]:
pickle.dump(result, open('./result.pkl', 'w'))


In [81]:
(pd.DataFrame(k).T*pd.Series(contri['SanDiego'])).sum(axis=1)

dw        0.0
fridge    0.0
hvac      0.0
mw        0.0
oven      0.0
wm        0.0
dtype: float64